In [1]:
%cd /app

/app


In [2]:
import argparse
import os
import sys

os.environ["XLA_PYTHON_CLIENT_PREALLOCATE"] = "false"

import torch
torch.multiprocessing.set_start_method('spawn')

import jax
from lob.encoding import Vocab, Message_Tokenizer

from lob import inference_no_errcorr as inference
from lob.init_train import init_train_state, load_checkpoint, load_metadata, load_args_from_checkpoint

from lob import inference_no_errcorr as inference
import lob.encoding as encoding
import preproc as preproc

import jax.numpy as jnp
import numpy as np

from pathlib import Path
import os

import pandas as pd
import plotly.graph_objs as go
import yaml

from filtration_utils import summary_table, build_zero_padded_series, plot_midprice_series_with_insertions, prepare_volatility_filtered_series, plot_midprice_series_with_mean_std

2025-08-05 10:15:49.346707: W external/xla/xla/service/gpu/nvptx_compiler.cc:718] The NVIDIA driver's CUDA version is 12.8 which is older than the ptxas CUDA version (12.9.41). Because the driver is older than the ptxas version, XLA is disabling parallel compilation, which may slow down compilation. You should update your NVIDIA driver or use the NVIDIA-provided CUDA forward compatibility packages.
2025-08-05 10:15:51.756943: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
# experiment_name = 'exp_92_20250702_215802'
experiment_name = 'exp_24_20250805_101039'


CONFIG_PATH = f"/app/data_saved/{experiment_name}/used_config.yaml"

# Load YAML config
with open(CONFIG_PATH, 'r') as f:
    config = yaml.safe_load(f)

# Extract values
num_insertions      = config["num_insertions"]
num_coolings        = config["num_coolings"]
midprice_step_size  = config["midprice_step_size"]
hist_msgs           = config["n_messages"]
n_gen_msgs          = config["n_gen_msgs"]
Direction           = config["DIRECTION_i"]

In [4]:
print(f"Aggressive {'buy' if Direction == 0 else 'sell'}\n")
print(f'num_insertions: {num_insertions}')
print(f'num_coolings: {num_coolings}')
print(f'midprice_step_size: {midprice_step_size}')
print(f'hist_msgs: {hist_msgs}')
print(f'n_gen_msgs: {n_gen_msgs}')

Aggressive buy

num_insertions: 20
num_coolings: 30
midprice_step_size: 1
hist_msgs: 500
n_gen_msgs: 50


In [5]:
hist_steps = hist_msgs // midprice_step_size       # 500
gen_steps = n_gen_msgs // midprice_step_size     # 50
gen_block = gen_steps + 1                        # 51

merged = summary_table(experiment_name)
x, all_series = build_zero_padded_series(hist_msgs, n_gen_msgs, midprice_step_size, merged)

print(merged)

      id                                        merged_data
0   2787  [897700, 897700, 897700, 897700, 897700, 89770...
1   4668  [887000, 887000, 887000, 887000, 887000, 88700...
2  14606  [867800, 867800, 867800, 867800, 867800, 86780...
3  16120  [872300, 872300, 872300, 872300, 872300, 87230...


In [6]:
fig = plot_midprice_series_with_insertions(
    merged,
    all_series,
    x,
    hist_steps,
    gen_block,
    num_insertions,
    num_coolings,
    n_gen_msgs,
    midprice_step_size
)
fig.show()

insertion positions: [551, 602, 653, 704, 755, 806, 857, 908, 959, 1010, 1061, 1112, 1163, 1214, 1265, 1316, 1367, 1418, 1469, 1520]
cooling positions:   [1571, 1622, 1673, 1724, 1775, 1826, 1877, 1928, 1979, 2030, 2081, 2132, 2183, 2234, 2285, 2336, 2387, 2438, 2489, 2540, 2591, 2642, 2693, 2744, 2795, 2846, 2897, 2948, 2999, 3050]


In [7]:
x, all_series, merged, hist_steps, gen_block = prepare_volatility_filtered_series(merged, hist_msgs, n_gen_msgs, midprice_step_size, volatility_cutoff=0.3)

      id     std_dev  max_abs_dev
0  16120  265.771216         1000
1   2787  221.881646          700
2  14606  130.907884          500
3   4668  148.356013          400

Before filtering: 4 samples

After filtering: 3 samples


In [8]:
fig, mean_series, std_series = plot_midprice_series_with_mean_std(
    merged=merged,
    all_series=all_series,
    x=x,
    hist_steps=hist_steps,
    gen_block=gen_block,
    num_insertions=num_insertions,
    num_coolings=num_coolings,
    n_gen_msgs=n_gen_msgs,
    midprice_step_size=midprice_step_size,
)
fig.show()

insertion positions: [551, 602, 653, 704, 755, 806, 857, 908, 959, 1010, 1061, 1112, 1163, 1214, 1265, 1316, 1367, 1418, 1469, 1520]
cooling positions:   [1571, 1622, 1673, 1724, 1775, 1826, 1877, 1928, 1979, 2030, 2081, 2132, 2183, 2234, 2285, 2336, 2387, 2438, 2489, 2540, 2591, 2642, 2693, 2744, 2795, 2846, 2897, 2948, 2999, 3050]


In [9]:
# np.save("/app/data_saved/exp_96_20250703_212149/exp_96_20250703_212149_mean.npy", mean_series)
# np.save("/app/data_saved/exp_96_20250703_212149/exp_96_20250703_212149_std.npy", std_series)

# ----------------------------------------------------

# ----------------------------------------------------

# ----------------------------------------------------

# OREDER-PLAYER

In [10]:
# import os, glob, re
# import numpy as np
# import pandas as pd

# # PRINT EACH NAME + SHAPE WHEN LOADING
# def build_and_merge(folder, batch_prefix, inp_prefix):
#     # STEP 1: load every .npy (shape (batch_size, time, feat)) into a DataFrame
#     files   = glob.glob(os.path.join(folder, "*.npy"))
#     rx_iter = re.compile(rf"{re.escape(batch_prefix)}_\[(.+)\]_iter_(\d+)\.npy$")
#     rx_inp  = re.compile(rf"{re.escape(inp_prefix)}_\[(.+)\]\.npy$")
#     rec = []
#     for f in files:
#         nm = os.path.basename(f)
#         m  = rx_iter.match(nm)
#         if m:
#             rng, itr = m.group(1).replace(" ", ""), int(m.group(2))
#         else:
#             m2 = rx_inp.match(nm)
#             if not m2:
#                 continue
#             rng, itr = m2.group(1).replace(" ", ""), 0
#         batch = np.load(f)  # shape (batch_size, time, features)

#         # ======== PRINT FILE NAME AND SHAPE =========
#         print(f"Loaded {nm} with shape {batch.shape}")
#         # ============================================

#         rec.append({"range": rng, "iteration": itr, "batch": batch})
#     df = pd.DataFrame(rec).sort_values(["range","iteration"]).reset_index(drop=True)

#     # STEP 2: parse the comma‐separated list of IDs into Python lists
#     df["ids"] = df["range"].str.split(",").apply(lambda L: [int(x) for x in L])

#     # explode each batch into one row per sample
#     rows = []
#     for _, r in df.iterrows():
#         for idx, sample_id in enumerate(r["ids"]):
#             single = r["batch"][idx]   # shape (time, features)
#             rows.append({
#                 "id":        sample_id,
#                 "iteration": r["iteration"],
#                 "data":      single
#             })
#     df_sorted = pd.DataFrame(rows).sort_values(["id","iteration"]).reset_index(drop=True)

#     # STEP 3: for each id, concatenate all its iterations end-to-end
#     merged = []
#     for id_val, grp in df_sorted.groupby("id", sort=True):
#         arrs = [row.data for _, row in grp.iterrows()]
#         big  = np.concatenate(arrs, axis=0)   # (sum_time, features)
#         merged.append({"id": id_val, "merged_data": big})
#     merged_df = pd.DataFrame(merged).sort_values("id").reset_index(drop=True)

#     return df, df_sorted, merged_df

# # — example usage —
# b_folder      = f"/app/data_saved/{experiment_name}/b_seq_gen_doubled"
# b_batch_pref  = "b_seq_gen_doubled_batch"
# b_inp_pref    = "b_seq_inp"
# _, b_sorted, b_merged = build_and_merge(b_folder, b_batch_pref, b_inp_pref)

# m_folder      = f"/app/data_saved/{experiment_name}/msgs_decoded_doubled"
# m_batch_pref  = "msgs_decoded_doubled_batch"
# m_inp_pref    = "m_seq_raw_inp"
# _, m_sorted, m_merged = build_and_merge(m_folder, m_batch_pref, m_inp_pref)

# # build your dicts
# b_dict = { int(r.id): np.array(r.merged_data) for _, r in b_merged.iterrows() }
# m_dict = { int(r.id): np.array(r.merged_data) for _, r in m_merged.iterrows() }

# # prepend zero‐row so that your interactive plot indexing from t=1…T works
# for d in (b_dict, m_dict):
#     for key, arr in d.items():
#         zero = np.zeros((1, arr.shape[1]), dtype=arr.dtype)
#         d[key] = np.vstack([zero, arr])

In [11]:
import os, glob, re
import numpy as np
import pandas as pd

def build_and_merge(folder, batch_prefix, inp_prefix):
    # STEP 1: load every .npy (shape (batch_size, time, feat)) into a DataFrame
    files   = glob.glob(os.path.join(folder, "*.npy"))
    rx_iter = re.compile(rf"{re.escape(batch_prefix)}_\[(.+)\]_iter_(\d+)\.npy$")
    rx_inp  = re.compile(rf"{re.escape(inp_prefix)}_\[(.+)\]\.npy$")
    rec = []
    for f in files:
        nm = os.path.basename(f)
        m  = rx_iter.match(nm)
        if m:
            rng, itr = m.group(1).replace(" ", ""), int(m.group(2))
        else:
            m2 = rx_inp.match(nm)
            if not m2:
                continue
            rng, itr = m2.group(1).replace(" ", ""), 0

        batch = np.load(f)  # shape (batch_size, time, features)
        print(f"Loaded {nm} with shape {batch.shape}")

        rec.append({"range": rng, "iteration": itr, "batch": batch})
    df = pd.DataFrame(rec).sort_values(["range","iteration"]).reset_index(drop=True)

    # STEP 2: parse the comma‐separated list of IDs into Python lists
    df["ids"] = df["range"].str.split(",").apply(lambda L: [int(x) for x in L])

    # explode each batch into one row per sample, с учётом slicing
    rows = []
    for _, r in df.iterrows():
        for idx, sample_id in enumerate(r["ids"]):
            single = r["batch"][idx]   # shape (time, features)

            # ====== здесь происходит нужный slice ======
            if r["iteration"] > 0:
                # для первой генерации — последние 51, для остальных — по 50
                n_keep = 51 if r["iteration"] == 1 else 50
                single = single[-n_keep:, :]
            # ============================================

            rows.append({
                "id":        sample_id,
                "iteration": r["iteration"],
                "data":      single
            })

    df_sorted = pd.DataFrame(rows).sort_values(["id","iteration"]).reset_index(drop=True)

    # STEP 3: для каждого id склеить его кусочки
    merged = []
    for id_val, grp in df_sorted.groupby("id", sort=True):
        arrs = [row.data for _, row in grp.iterrows()]
        big  = np.concatenate(arrs, axis=0)   # (sum_time, features)
        merged.append({"id": id_val, "merged_data": big})
    merged_df = pd.DataFrame(merged).sort_values("id").reset_index(drop=True)

    return df, df_sorted, merged_df

b_folder      = f"/app/data_saved/{experiment_name}/b_seq_gen_doubled"
b_batch_pref  = "b_seq_gen_doubled_batch"
b_inp_pref    = "b_seq_inp"

m_folder      = f"/app/data_saved/{experiment_name}/msgs_decoded_doubled"
m_batch_pref  = "msgs_decoded_doubled_batch"
m_inp_pref    = "m_seq_raw_inp"

_, b_sorted, b_merged = build_and_merge(b_folder, b_batch_pref, b_inp_pref)
_, m_sorted, m_merged = build_and_merge(m_folder, m_batch_pref, m_inp_pref)

# Словари
b_dict = { int(r.id): np.array(r.merged_data) for _, r in b_merged.iterrows() }
m_dict = { int(r.id): np.array(r.merged_data) for _, r in m_merged.iterrows() }

# prepend zero‐row для интерактивного индекса
for d in (b_dict, m_dict):
    for key, arr in d.items():
        zero = np.zeros((1, arr.shape[1]), dtype=arr.dtype)
        d[key] = np.vstack([zero, arr])

Loaded b_seq_gen_doubled_batch_[14606, 16120, 2787, 4668]_iter_6.npy with shape (4, 501, 501)
Loaded b_seq_inp_[14606, 16120, 2787, 4668].npy with shape (4, 500, 501)
Loaded b_seq_gen_doubled_batch_[14606, 16120, 2787, 4668]_iter_4.npy with shape (4, 501, 501)
Loaded b_seq_gen_doubled_batch_[14606, 16120, 2787, 4668]_iter_2.npy with shape (4, 501, 501)
Loaded b_seq_gen_doubled_batch_[14606, 16120, 2787, 4668]_iter_1.npy with shape (4, 501, 501)
Loaded b_seq_gen_doubled_batch_[14606, 16120, 2787, 4668]_iter_3.npy with shape (4, 501, 501)
Loaded b_seq_gen_doubled_batch_[14606, 16120, 2787, 4668]_iter_7.npy with shape (4, 501, 501)
Loaded b_seq_gen_doubled_batch_[14606, 16120, 2787, 4668]_iter_5.npy with shape (4, 501, 501)
Loaded msgs_decoded_doubled_batch_[14606, 16120, 2787, 4668]_iter_5.npy with shape (4, 51, 14)
Loaded msgs_decoded_doubled_batch_[14606, 16120, 2787, 4668]_iter_7.npy with shape (4, 51, 14)
Loaded m_seq_raw_inp_[14606, 16120, 2787, 4668].npy with shape (4, 500, 14)
Loa

In [12]:
# m_dict[14606].shape

In [13]:
b_merged

,id,merged_data
0,2787,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
1,4668,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
2,14606,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
3,16120,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."


In [14]:
# b_dict[14606].shape

In [15]:
import numpy as np
import pandas as pd
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import ipywidgets as widgets
from IPython.display import display

def interactive_lob_plot(b_seq_inp, msg_seq_raw):
    # allow DataFrame or dict
    if isinstance(b_seq_inp, pd.DataFrame):
        b_seq_inp = {int(r.id): np.array(r.merged_data) for _,r in b_seq_inp.iterrows()}
    if isinstance(msg_seq_raw, pd.DataFrame):
        msg_seq_raw = {int(r.id): np.array(r.merged_data) for _,r in msg_seq_raw.iterrows()}

    # controls
    id_dd       = widgets.Dropdown(options=sorted(b_seq_inp.keys()), description="Sample ID:")
    time_slider = widgets.IntSlider(min=1, max=1, step=1, description="t:")
    btn_prev    = widgets.Button(description="←")
    btn_next    = widgets.Button(description="→")
    msg_box     = widgets.HTML()

    # figure with two subplots
    fig = make_subplots(rows=1, cols=2, subplot_titles=["Book state t–1","Book state t"])
    fig.add_trace(go.Bar(x=[],y=[]), row=1,col=1)
    fig.add_trace(go.Bar(x=[],y=[]), row=1,col=2)
    fig.update_layout(width=800, height=400, showlegend=False, template='plotly_white')
    fig_widget = go.FigureWidget(fig)

    def update_slider_range(*_):
        arr = b_seq_inp[id_dd.value]
        # now slider runs 1…(T−1)
        time_slider.min = 1
        time_slider.max = arr.shape[0] - 1
        time_slider.value = 1

    def update_plot(*_):
        sid = id_dd.value
        t   = time_slider.value
        arr = b_seq_inp[sid]
        msgs= msg_seq_raw[sid]

        # take book states at t−1 and t
        s0 = arr[t-1, 240:263]
        s1 = arr[t,   240:263]

        # s0 = arr[t-1]
        # s1 = arr[t]


        diff = abs(s1) - abs(s0)
        x = np.arange(len(s0)) - len(s0)//2

        with fig_widget.batch_update():
            fig_widget.data = []
            fig_widget.add_bar(x=x, y=s0, row=1, col=1, marker_color='orange')
            
            colors = ['orange' if abs(d)<1e-8 else ('red' if d>0 else 'blue') for d in diff]
                        
            fig_widget.add_bar(x=x, y=s1, row=1, col=2, marker_color=colors)
            fig_widget.layout.annotations[0].text = f"Book state {t-1}"
            fig_widget.layout.annotations[1].text = f"Book state {t}"

        # show message at index t
        m = msgs[t].astype(int)
        # fields: [0]=timestamp, [1]=etype, [2]=dir, [3]=abspr, [4]=relpr, [5]=size, …
        et, dr, abspr, relpr, sz = m[1], m[2], m[3], m[4], m[5]
        et_map = {1:"Limit",2:"PartialCancel",3:"Delete",4:"Execution"}
        dr_map = {1:"Buy",0:"Sell"}
        info = (
            f"{et_map.get(et,'?')} • "
            f"{dr_map.get(dr,'?')} • "
            f"abs={abspr} • rel={relpr} • size={sz}"
        )
        msg_box.value = f"<b>{info}<br></b>raw:{m.tolist()}"

    def on_prev(b):
        if time_slider.value>time_slider.min:
            time_slider.value -= 1
    def on_next(b):
        if time_slider.value<time_slider.max:
            time_slider.value += 1

    # wire up events
    id_dd.observe(lambda c: update_slider_range(), names='value')
    time_slider.observe(lambda c: update_plot(), names='value')
    btn_prev.on_click(on_prev)
    btn_next.on_click(on_next)

    # initial draw
    update_slider_range()
    update_plot()

    display(widgets.HBox([id_dd, btn_prev, btn_next, time_slider]))
    display(fig_widget, msg_box)


interactive_lob_plot(b_dict, m_dict)

FigureWidget({
    'data': [{'marker': {'color': 'orange'},
              'type': 'bar',
              'uid': '3e34705f-4b62-4de3-b712-9d6435298b95',
              'x': {'bdata': '9fb3+Pn6+/z9/v8AAQIDBAUGBwgJCgs=', 'dtype': 'i1'},
              'xaxis': 'x',
              'y': {'bdata': ('AAAAAAAAAAAAAAAAAAAAAAAAAAAAAA' ... 'AAAAAAAAAAAAAAAAAAAAAAAAAAAAA='),
                    'dtype': 'f4'},
              'yaxis': 'y'},
             {'marker': {'color': [orange, red, red, red, red, red, red, red, red,
                                   red, red, orange, red, red, red, red, red, red,
                                   red, red, red, red, orange]},
              'type': 'bar',
              'uid': 'bb22aa57-c4b6-43a0-8587-a109997483ac',
              'x': {'bdata': '9fb3+Pn6+/z9/v8AAQIDBAUGBwgJCgs=', 'dtype': 'i1'},
              'xaxis': 'x2',
              'y': {'bdata': ('AAAAAFCNlz4JrDw/2KPwPobrkT/azj' ... 'K/tcj2vipcz77BymG/NDOzvgAAAAA='),
                    'dtype': 'f4'},
     

HTML(value='<b>Limit • Buy • abs=897600 • rel=-1 • size=22<br></b>raw:[693834242, 1, 1, 897600, -1, 22, 0, 0, …

In [16]:
def show_execution_indices(m_dict):
    """
    For each sample_id in m_dict, prints indices t where etype == 4 (Execution).
    """
    for sid, msgs in m_dict.items():
        etypes = msgs[:, 1]
        exec_indices = np.where(etypes == 4)[0]
        if len(exec_indices) > 0:
            print(f"Sample ID {sid}: Execution at indices {exec_indices.tolist()}")
        else:
            print(f"Sample ID {sid}: No Executions found")

show_execution_indices(m_dict)

Sample ID 2787: Execution at indices [149, 150, 151, 200, 208, 214, 221, 304, 340, 341, 537, 544, 545, 546, 551, 601, 651, 701, 751, 801, 851]
Sample ID 4668: Execution at indices [184, 251, 270, 449, 551, 601, 651, 701, 751, 800, 801, 851]
Sample ID 14606: Execution at indices [104, 105, 317, 321, 323, 324, 350, 376, 377, 378, 437, 522, 551, 572, 574, 575, 577, 582, 593, 594, 595, 597, 601, 622, 624, 625, 627, 632, 647, 651, 661, 667, 672, 674, 675, 677, 682, 693, 694, 695, 696, 697, 701, 711, 717, 722, 724, 725, 727, 732, 743, 744, 745, 747, 751, 772, 774, 775, 777, 782, 793, 794, 795, 797, 801, 822, 824, 825, 827, 832, 843, 844, 845, 847, 851]
Sample ID 16120: Execution at indices [78, 79, 80, 105, 106, 107, 385, 551, 601, 651, 701, 722, 723, 724, 725, 751, 801, 851]


In [17]:
import numpy as np

# Явные пути к файлам
inp_file = "/app/data_saved/exp_67_20250701_164001/msgs_decoded_doubled/m_seq_raw_inp_[14606, 16120].npy"
iter_file = "/app/data_saved/exp_67_20250701_164001/msgs_decoded_doubled/msgs_decoded_doubled_batch_[14606, 16120]_iter_1.npy"
before_insert_file = "/app/data_saved/exp_67_20250701_164001/msgs_decoded_doubled/msgs_decoded_before_insert_batch_[14606, 16120]_iter_1.npy"

# Загрузка
arr_inp = np.load(inp_file)
arr_iter = np.load(iter_file)

# id = 2787 находится в первой позиции [0]
row_inp = arr_inp[0, -1, :]
row_iter = arr_iter[0, 0, :]

# Печать
print(row_inp)
print(row_iter)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:9                                                                                    │
│                                                                                                  │
│    6 before_insert_file = "/app/data_saved/exp_67_20250701_164001/msgs_decoded_doubled/msgs_d    │
│    7                                                                                             │
│    8 # Загрузка                                                                                  │
│ ❱  9 arr_inp = np.load(inp_file)                                                                 │
│   10 arr_iter = np.load(iter_file)                                                               │
│   11                                                                                             │
│   12 # id = 2787 находится в первой позиции [0]                                                  │
│                                                                                                  │
│ /opt/conda/envs/myenv/lib/python3.12/site-packages/numpy/lib/npyio.py:427 in load                │
│                                                                                                  │
│    424 │   │   │   fid = file                                                                    │
│    425 │   │   │   own_fid = False                                                               │
│    426 │   │   else:                                                                             │
│ ❱  427 │   │   │   fid = stack.enter_context(open(os_fspath(file), "rb"))                        │
│    428 │   │   │   own_fid = True                                                                │
│    429 │   │                                                                                     │
│    430 │   │   # Code to distinguish from NumPy binary files and pickles.                        │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
FileNotFoundError: [Errno 2] No such file or directory: 
'/app/data_saved/exp_67_20250701_164001/msgs_decoded_doubled/m_seq_raw_inp_[14606, 16120].npy'

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display

# === 1️⃣ Указываю ТВОИ актуальные пути ===
inp_file = "/app/data_saved/exp_73_20250701_210502/b_seq_gen_doubled/b_seq_inp_[14606, 16120].npy"
iter_file = "/app/data_saved/exp_73_20250701_210502/b_seq_gen_doubled/b_seq_gen_doubled_batch_[14606, 16120]_iter_1.npy"
before_insert_file = "/app/data_saved/exp_73_20250701_210502/b_seq_gen_doubled/b_seq_gen_before_insert_batch_[14606, 16120]_iter_1.npy"

# === 2️⃣ Загружаем массивы ===
arr_inp = np.load(inp_file)
arr_iter = np.load(iter_file)
arr_before_insert = np.load(before_insert_file)

feat_range = slice(240, 263)
x = np.arange(240, 263) - ((263 - 240) // 2)

# inp
batch_size_inp, T_inp, _ = arr_inp.shape
# iter_1
batch_size_iter, T_iter, _ = arr_iter.shape
# before_insert
batch_size_bef, T_bef, _ = arr_before_insert.shape

# === 3️⃣ Создаём виджеты ===
sample_slider_inp = widgets.IntSlider(min=0, max=batch_size_inp-1, step=1, value=0, description="sample_inp")
t_slider_inp = widgets.IntSlider(min=0, max=T_inp-1, step=1, value=T_inp-1, description="t_inp")

sample_slider_iter = widgets.IntSlider(min=0, max=batch_size_iter-1, step=1, value=0, description="sample_iter")
t_slider_iter = widgets.IntSlider(min=0, max=T_iter-1, step=1, value=0, description="t_iter")

sample_slider_bef = widgets.IntSlider(min=0, max=batch_size_bef-1, step=1, value=0, description="sample_bef")
t_slider_bef = widgets.IntSlider(min=0, max=T_bef-1, step=1, value=0, description="t_bef")

# === 4️⃣ Функция отрисовки ===
def plot_all(sample_inp, t_inp, sample_iter, t_iter, sample_bef, t_bef):
    row_inp = arr_inp[sample_inp, t_inp, feat_range]
    row_iter = arr_iter[sample_iter, t_iter, feat_range]
    row_bef = arr_before_insert[sample_bef, t_bef, feat_range]
    
    fig, axs = plt.subplots(1, 3, figsize=(18, 4), sharey=True)

    axs[0].bar(x, row_inp, color='orange')
    axs[0].set_title(f"inp[{t_inp}] | sample {sample_inp}")
    axs[0].set_xlabel("Level rel to mid")
    axs[0].grid(True, alpha=0.3)

    axs[1].bar(x, row_iter, color='orange')
    axs[1].set_title(f"iter_1[{t_iter}] | sample {sample_iter}")
    axs[1].set_xlabel("Level rel to mid")
    axs[1].grid(True, alpha=0.3)

    axs[2].bar(x, row_bef, color='orange')
    axs[2].set_title(f"before_insert[{t_bef}] | sample {sample_bef}")
    axs[2].set_xlabel("Level rel to mid")
    axs[2].grid(True, alpha=0.3)

    axs[0].set_ylabel("Normalized volume")
    plt.show()

# === 5️⃣ Отображаем виджеты и связываем с функцией ===
ui = widgets.VBox([
    widgets.HBox([sample_slider_inp, t_slider_inp]),
    widgets.HBox([sample_slider_iter, t_slider_iter]),
    widgets.HBox([sample_slider_bef, t_slider_bef]),
])

out = widgets.interactive_output(
    plot_all,
    {
        'sample_inp': sample_slider_inp,
        't_inp': t_slider_inp,
        'sample_iter': sample_slider_iter,
        't_iter': t_slider_iter,
        'sample_bef': sample_slider_bef,
        't_bef': t_slider_bef,
    }
)

display(ui, out)

In [ ]:
import numpy as np
import ipywidgets as widgets
from IPython.display import display

# === 1️⃣ Указываю актуальные пути ===
inp_file = "/app/data_saved/exp_67_20250701_164001/msgs_decoded_doubled/m_seq_raw_inp_[14606, 16120].npy"
iter_file = "/app/data_saved/exp_67_20250701_164001/msgs_decoded_doubled/msgs_decoded_doubled_batch_[14606, 16120]_iter_1.npy"
before_insert_file = "/app/data_saved/exp_67_20250701_164001/msgs_decoded_doubled/msgs_decoded_before_insert_batch_[14606, 16120]_iter_1.npy"

# === 2️⃣ Загружаем массивы ===
arr_inp = np.load(inp_file)
arr_iter = np.load(iter_file)
arr_before_insert = np.load(before_insert_file)

batch_size_inp, T_inp, _ = arr_inp.shape
batch_size_iter, T_iter, _ = arr_iter.shape
batch_size_bef, T_bef, _ = arr_before_insert.shape

# === 3️⃣ Виджеты ===
sample_slider_inp = widgets.IntSlider(min=0, max=batch_size_inp-1, step=1, value=0, description="sample_inp")
t_slider_inp = widgets.IntSlider(min=0, max=T_inp-1, step=1, value=0, description="t_inp")

sample_slider_iter = widgets.IntSlider(min=0, max=batch_size_iter-1, step=1, value=0, description="sample_iter")
t_slider_iter = widgets.IntSlider(min=0, max=T_iter-1, step=1, value=0, description="t_iter")

sample_slider_bef = widgets.IntSlider(min=0, max=batch_size_bef-1, step=1, value=0, description="sample_bef")
t_slider_bef = widgets.IntSlider(min=0, max=T_bef-1, step=1, value=0, description="t_bef")

# === 4️⃣ Функция вывода сообщений ===
def print_all(sample_inp, t_inp, sample_iter, t_iter, sample_bef, t_bef):
    msg_inp = arr_inp[sample_inp, t_inp]
    msg_iter = arr_iter[sample_iter, t_iter]
    msg_bef = arr_before_insert[sample_bef, t_bef]

    print(f"📌 inp[{t_inp}] | sample {sample_inp}:\n{msg_inp.astype(int).tolist()}\n")
    print(f"📌 iter_1[{t_iter}] | sample {sample_iter}:\n{msg_iter.astype(int).tolist()}\n")
    print(f"📌 before_insert[{t_bef}] | sample {sample_bef}:\n{msg_bef.astype(int).tolist()}\n")

# === 5️⃣ Отображение виджетов и связка ===
ui = widgets.VBox([
    widgets.HBox([sample_slider_inp, t_slider_inp]),
    widgets.HBox([sample_slider_iter, t_slider_iter]),
    widgets.HBox([sample_slider_bef, t_slider_bef]),
])

out = widgets.interactive_output(
    print_all,
    {
        'sample_inp': sample_slider_inp,
        't_inp': t_slider_inp,
        'sample_iter': sample_slider_iter,
        't_iter': t_slider_iter,
        'sample_bef': sample_slider_bef,
        't_bef': t_slider_bef,
    }
)

display(ui, out)

In [ ]:
import numpy as np
import plotly.graph_objects as go

# Load data
mean_92 = np.load("/app/data_saved/exp_92_20250702_215802/exp_92_20250702_215802_mean.npy")
std_92 = np.load("/app/data_saved/exp_92_20250702_215802/exp_92_20250702_215802_std.npy")
mean_96 = np.load("/app/data_saved/exp_96_20250703_212149/exp_96_20250703_212149_mean.npy")
std_96 = np.load("/app/data_saved/exp_96_20250703_212149/exp_96_20250703_212149_std.npy")

mean_96 = -mean_96
# std_96 = -std_96

x_92 = np.arange(len(mean_92))
x_96 = np.arange(len(mean_96))

min_y = min(np.min(mean_92 - std_92), np.min(mean_96 - std_96))
max_y = max(np.max(mean_92 + std_92), np.max(mean_96 + std_96))

fig = go.Figure()

# exp_92 (now RED)
fig.add_trace(go.Scatter(
    x=np.concatenate([x_92, x_92[::-1]]),
    y=np.concatenate([mean_92 + std_92, (mean_92 - std_92)[::-1]]),
    fill='toself',
    fillcolor='rgba(200, 0, 0, 0.2)',
    line=dict(color='rgba(255,255,255,0)'),
    hoverinfo="skip",
    name="exp_92 ±1 std"
))
fig.add_trace(go.Scatter(
    x=x_92,
    y=mean_92,
    mode='lines',
    line=dict(color='red', width=2),
    name="exp_92 mean"
))

# exp_96 (now BLUE)
fig.add_trace(go.Scatter(
    x=np.concatenate([x_96, x_96[::-1]]),
    y=np.concatenate([mean_96 + std_96, (mean_96 - std_96)[::-1]]),
    fill='toself',
    fillcolor='rgba(0, 100, 200, 0.2)',
    line=dict(color='rgba(255,255,255,0)'),
    hoverinfo="skip",
    name="exp_96 ±1 std"
))
fig.add_trace(go.Scatter(
    x=x_96,
    y=mean_96,
    mode='lines',
    line=dict(color='blue', width=2),
    name="exp_96 mean"
))

# Vertical dashed lines
fig.add_shape(type="line", x0=500, x1=500, y0=min_y, y1=max_y,
              line=dict(color="black", dash="dash"))
fig.add_shape(type="line", x0=1000, x1=1000, y0=min_y, y1=max_y,
              line=dict(color="blue", dash="dash"))
fig.add_shape(type="line", x0=1500, x1=1500, y0=min_y, y1=max_y,
              line=dict(color="red", dash="dash"))

# Horizontal zero line
fig.add_shape(
    type="line",
    x0=0,
    x1=max(len(x_92), len(x_96)),
    y0=0,
    y1=0,
    line=dict(color="gray")
)

# Layout
fig.update_layout(
    title="Aggresive buy (red, 144 samples, 20 buys, 30 cooldown) VS sell (blue, 98 samples, 10 buys & 40 cooldown)",
    xaxis_title="Message Index",
    yaxis_title="Midprice (zeroed at first price)",
    template="plotly_white",
    height=800,
    width=1000,
    legend=dict(orientation="h", y=-0.2),
    hovermode="x unified",
)

fig.show()